# Customer Analysis Cleaning 



<i> You are working as an analyst for an auto insurance company. The company has collected some data about its customers including their demographics, education, employment, policy details, vehicle information on which insurance policy is, and claim amounts. You will help the senior management with some business questions that will help them to better understand their customers, improve their services, and improve profitability. </i>
    
 **Notebook operations summary:**
 <p> We import three data files in cvs format and clean them. <br>
     The cleaning process includes: </p>

        - Renaming columns (to ensure consistent concatenation and eliminate column redundancy)
        - Replacing null data entries (zeros and nans) by mean values
        - Standardizing string entries
        - Removing duplicate records

        

In [5]:
import numpy as np
import pandas as pd
from skimpy import clean_columns
from nltk.metrics import edit_distance
pd.options.display.max_rows = 100

## Helper functions

In [6]:
class Cleaner:
    def __init__(self, df):

        self.cdf = clean_columns(df,case='snake')
        self.numerics_df =  self.cdf.select_dtypes(include=[np.number])
        self.numeric_cols = self.numerics_df.columns.values
        self.cats_df =   self.cdf.select_dtypes(include=object)
        self.cat_cols = self.cats_df.columns.values

    def drop_dups(self):
        df = self.cdf
        df_dedupped = df.drop_duplicates()
        # there were duplicate rows
        print('Dropping {} duplicates'.format(df.shape[0] - df_dedupped.shape[0]))
        df = df_dedupped

    def drop_select_dups(self, keys):
        #df.fillna(-999).groupby(keys)['id'].count().sort_values(ascending=False).head(20)
        self.cdf = self.cdf.drop_duplicates(subset=keys)

    def show_missing(self):
        for col in self.cdf.columns:
            pct_missing = np.mean(self.cdf[col].isnull())
            print('{} - {}%'.format(col, round(pct_missing*100)))

    def drop_missing(self, replace=False):
        df = self.cdf
        # first create missing indicator for features with missing data
        for col in df.columns:
            missing = df[col].isnull()
            num_missing = np.sum(missing)
            if num_missing > 0:
                print('created missing indicator for: {}'.format(col))
                df['{}_ismissing'.format(col)] = missing

            if replace == True:
                # categorical
                if col in set(self.numeric_cols):
                    print('Numerical column '+col+' has missing values.')
                    filler = input('Input a filler value (e.g. -999).')
                    df[col] = df[col].fillna(filler).astype(int)
                else:
                    print('Categorical column '+col+' has missing values.')
                    filler = input('Input a filler value (e.g. _MISSING_). ')
                    df[col] = df[col].fillna(filler)

        if replace == False:
            from math import ceil
            cut_off = ceil(0.50*len(df.columns)) # if a row is missing more than 10% of its entries drop it
            ismissing_cols = [col for col in df.columns if 'ismissing' in col]
            df['num_missing'] = df[ismissing_cols].sum(axis=1)
            # drop rows with a lot of missing values.
            ind_missing = df[df['num_missing'] > cut_off].index
            df = df.drop(ind_missing, axis=0)

    def impute_numerics(self):
        df = self.cdf
        for col in self.numeric_cols:
            missing = df[col].isnull()
            num_missing = np.sum(missing)

            if num_missing > 0:  # only do the imputation for the columns that have missing values.
                print('imputing missing values for: {}'.format(col))
                #df['{}_ismissing'.format(col)] = missing
                med = df[col].median()
                df[col] = df[col].fillna(med)

    def impute_cats(self):
        df = self.cdf
        for col in self.cat_cols:
            missing = df[col].isnull()
            num_missing = np.sum(missing)

            if num_missing > 0:  # only do the imputation for the columns that have missing values.
                print('imputing missing values for: {}'.format(col))
                #df['{}_ismissing'.format(col)] = missing
                top = df[col].describe()['top'] # impute with the most frequent value.
                df[col] = df[col].fillna(top)

    def redundants(self):
        df = self.cdf
        num_rows = len(df.index)
        low_information_cols = [] #

        for col in df.columns:
            cnts = df[col].value_counts(dropna=False)
            top_pct = (cnts/num_rows).iloc[0]

            if top_pct > 0.95:
                low_information_cols.append(col)
                print('{0}: {1:.5f}%'.format(col, top_pct*100))
                print(cnts)
                print()


    def col_rename(self, dict_rules):
        ''' rename columns in data frame using a dictionary of rules  '''
        if isinstance(dict_rules,dict):
            self.cdf.rename(columns = dict_rules, inplace=True)
        else:
            raise TypeError

    def expand_timestamp(self, time_colname):
        df = self.cdf
        df['timestamp_dt'] = pd.to_datetime(df[time_colname], format='%Y-%m-%d')
        df['year'] = df['timestamp_dt'].dt.year
        df['month'] = df['timestamp_dt'].dt.month
        df['weekday'] = df['timestamp_dt'].dt.weekday

        print(df['year'].value_counts(dropna=False))
        print()
        print(df['month'].value_counts(dropna=False))

    def get_typos(self,colname, word):
        tmp = pd.DataFrame([])
        tmp[colname] = self.cdf[colname]
        cn = 'distance_'+word
        tmp[cn] = self.cdf[colname].map(lambda x: edit_distance(x, word))
        return tmp.value_counts()

    def fix_typos(self,colname, word, dist=2):
        tmp = self.cdf.copy()
        tmp[colname] = self.cdf[colname]
        cn = 'distance_'+word
        tmp[cn] = self.cdf[colname].map(lambda x: edit_distance(x, word))
        msk = tmp[cn] <= dist
        self.cdf.loc[msk, colname] = word

    def row_string_replace(self, colname, replacement_dict):
        self.cdf[colname] = self.cdf[colname].replace(replacement_dict)



In [7]:
def print_unique(df,col):
    ''' gives a list of unique values in a field '''
    return df[col].unique()

def col_rename(df, dict_rules, ip=True):
    ''' rename columns in data frame using a dictionary of rules  '''
    if isinstance(df,pd.core.frame.DataFrame) and isinstance(dict_rules,dict):
        z = df.rename(columns = dict_rules, inplace=ip)
    else:
        raise TypeError
    return z

def fill_nans_with_means(df,col):
    ''' fills nans in column with median '''
    return df[col].fillna(c_df.income.mean()).round()

def make_lower(df, col):
    '''lower the case of record in a field '''
    return df[col].str.lower()

def lower_case_column_names(df):
    ''' make columns lower case '''
    if isinstance(df,pd.core.frame.DataFrame):
        df.columns=[i.lower() for i in df.columns]
    else:
        raise TypeError
    return df

def strip_char(df,col,char):
    ''' strips a char and rounds '''
    return list(map(lambda x: round(float(x.strip(char))/100,0)\
                                          if type(x)==str else round(x,0),df[col]))

def record_str_replace(df, col, dict_rules):
    return df[col].replace(dict_rules) # additional replacement

def get_between_slash_and_join(df,col):
    ''' returns middle entry in a string formatted by "a/b/cc" and avoids nans.
        (Tailored to the vehicles data set)  '''
    res  = list(map(lambda x : int(x[2]) if type(x)==str else x, df[col]))
    return res

def state(old_names, new_names):
    ''' renames states '''
    return c_df['st'].replace(old_names, new_names)

## Data cleaning

In [8]:
file1 = pd.read_csv('Data/file1.csv')
file2 = pd.read_csv('Data/file2.csv')
file3 = pd.read_csv('Data/file3.csv')

In [9]:
a_ll_columns = set(list(file1.columns) + list(file2.columns) + list(file3.columns))
a_ll_columns

{'Customer',
 'Customer Lifetime Value',
 'Education',
 'GENDER',
 'Gender',
 'Income',
 'Monthly Premium Auto',
 'Number of Open Complaints',
 'Policy Type',
 'ST',
 'State',
 'Total Claim Amount',
 'Vehicle Class'}

In [10]:
# rename State to ST before concat to avoid redundant columns
col_rename(file3, {'State': 'ST'}) 

In [11]:
files = [file1,file2,file3] # aggregate files
files = list(map(lower_case_column_names,files)) # make the headers lowercase
c_df  = pd.concat(files) # concat the data into a pandas frame
c_df  = c_df.drop(labels=['customer'], axis=1) # drop customer label
c_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12074 entries, 0 to 7069
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   st                         9137 non-null   object 
 1   gender                     9015 non-null   object 
 2   education                  9137 non-null   object 
 3   customer lifetime value    9130 non-null   object 
 4   income                     9137 non-null   float64
 5   monthly premium auto       9137 non-null   float64
 6   number of open complaints  9137 non-null   object 
 7   policy type                9137 non-null   object 
 8   vehicle class              9137 non-null   object 
 9   total claim amount         9137 non-null   float64
dtypes: float64(3), object(7)
memory usage: 1.0+ MB


In [12]:
c_df['customer lifetime value'] = strip_char(c_df,'customer lifetime value','%')

In [13]:
clnr = Cleaner(c_df)


6 column names have been cleaned

In [14]:
clnr.cdf.income.value_counts()

0.0        2294
95697.0      13
26876.0       8
61108.0       8
10621.0       8
           ... 
26599.0       1
75774.0       1
85412.0       1
42968.0       1
21941.0       1
Name: income, Length: 5655, dtype: int64

In [52]:
clnr.drop_dups() # drop duplicates

Dropping 3192 duplicates


In [53]:
#clnr.drop_missing()  #

In [54]:
clnr.impute_numerics() # replace nans by median
clnr.impute_cats() # replace nans by mode


imputing missing values for: customer_lifetime_value
imputing missing values for: income
imputing missing values for: monthly_premium_auto
imputing missing values for: total_claim_amount
imputing missing values for: st
imputing missing values for: gender
imputing missing values for: education
imputing missing values for: number_of_open_complaints
imputing missing values for: policy_type
imputing missing values for: vehicle_class


In [55]:
clnr.get_typos('gender', 'female')

gender  distance_female
F       6                  7619
M       6                  4368
Male    3                    40
female  0                    30
Femal   2                    17
dtype: int64

In [56]:
clnr.fix_typos('gender','male')
clnr.fix_typos('gender','female')

In [57]:
clnr.cdf.gender.value_counts()

F         7619
M         4368
female      87
Name: gender, dtype: int64

In [58]:
clnr.cdf

,st,gender,education,customer_lifetime_value,income,monthly_premium_auto,number_of_open_complaints,policy_type,vehicle_class,total_claim_amount
0,Washington,F,Master,77.15,0.0,1000.0,1/0/00,Personal Auto,Four-Door Car,2.704934
1,Arizona,F,Bachelor,6979.54,0.0,94.0,1/0/00,Personal Auto,Four-Door Car,1131.464935
2,Nevada,F,Bachelor,12887.43,48767.0,108.0,1/0/00,Personal Auto,Two-Door Car,566.472247
3,California,M,Bachelor,7645.86,0.0,106.0,1/0/00,Corporate Auto,SUV,529.881344
4,Washington,M,High School or Below,5363.08,36357.0,68.0,1/0/00,Personal Auto,Four-Door Car,17.269323
...,...,...,...,...,...,...,...,...,...,...
7065,California,M,Bachelor,234.06,71941.0,73.0,0,Personal Auto,Four-Door Car,198.234764
7066,California,F,College,30.97,21604.0,79.0,0,Corporate Auto,Four-Door Car,379.200000
7067,California,M,Bachelor,81.64,0.0,85.0,3,Corporate Auto,Four-Door Car,790.784983
7068,California,M,College,75.24,21941.0,96.0,0,Personal Auto,Four-Door Car,691.200000


In [59]:
#cc_df.col_rename({'st':'state'})

In [60]:
clnr.redundants() # print features have overwhelmingly the same values

In [61]:
clnr.cdf.gender.value_counts()


F         7619
M         4368
female      87
Name: gender, dtype: int64

In [68]:
clnr.row_string_replace('gender', {'female':'F'})

In [69]:
state_old = print_unique(clnr.cdf,'st')
state_old = list(state_old)
state_old

['East Region', 'Central', 'West Region', 'Northeast Region']

In [70]:
state_new = ['Washington',
 'Arizona',
 'Nevada',
 'California',
 'Oregon',
 'California',
 'Arizona',
 'Washington', 'Unknown']
state_replace_rules = dict(zip(state_old,state_new))
clnr.row_string_replace('st',state_replace_rules)
print_unique(clnr.cdf,'st')

array(['Washington', 'Arizona', 'Nevada', 'California'], dtype=object)

In [73]:
clnr.cdf['number of open complaints'] = get_between_slash_and_join(clnr.cdf,'number of open complaints')

In [74]:
clnr.cdf

,st,gender,education,customer_lifetime_value,income,monthly_premium_auto,number_of_open_complaints,policy_type,vehicle_class,total_claim_amount,number of open complaints
0,Washington,F,Master,77.15,0.0,1000.0,1/0/00,Personal Auto,Four-Door Car,2.704934,0.0
1,Arizona,F,Bachelor,6979.54,0.0,94.0,1/0/00,Personal Auto,Four-Door Car,1131.464935,0.0
2,Arizona,F,Bachelor,12887.43,48767.0,108.0,1/0/00,Personal Auto,Two-Door Car,566.472247,0.0
3,Nevada,M,Bachelor,7645.86,0.0,106.0,1/0/00,Corporate Auto,SUV,529.881344,0.0
4,Washington,M,High School or Below,5363.08,36357.0,68.0,1/0/00,Personal Auto,Four-Door Car,17.269323,0.0
...,...,...,...,...,...,...,...,...,...,...,...
7065,Nevada,M,Bachelor,234.06,71941.0,73.0,0,Personal Auto,Four-Door Car,198.234764,0.0
7066,Nevada,F,College,30.97,21604.0,79.0,0,Corporate Auto,Four-Door Car,379.200000,0.0
7067,Nevada,M,Bachelor,81.64,0.0,85.0,3,Corporate Auto,Four-Door Car,790.784983,3.0
7068,Nevada,M,College,75.24,21941.0,96.0,0,Personal Auto,Four-Door Car,691.200000,0.0


In [75]:
c_df['number of open complaints'] = get_between_slash_and_join(c_df,'number of open complaints')
c_df

,st,gender,education,customer lifetime value,income,monthly premium auto,number of open complaints,policy type,vehicle class,total claim amount
0,East Region,NaN,Master,NaN,0.0,1000.0,0.0,Personal Auto,Four-Door Car,2.704934
1,Central,F,Bachelor,6979.54,0.0,94.0,0.0,Personal Auto,Four-Door Car,1131.464935
2,Central,F,Bachelor,12887.43,48767.0,108.0,0.0,Personal Auto,Two-Door Car,566.472247
3,West Region,M,Bachelor,7645.86,0.0,106.0,0.0,Corporate Auto,SUV,529.881344
4,East Region,M,High School or Below,5363.08,36357.0,68.0,0.0,Personal Auto,Four-Door Car,17.269323
...,...,...,...,...,...,...,...,...,...,...
7065,West Region,M,Bachelor,234.06,71941.0,73.0,0.0,Personal Auto,Four-Door Car,198.234764
7066,West Region,F,College,30.97,21604.0,79.0,0.0,Corporate Auto,Four-Door Car,379.200000
7067,West Region,M,Bachelor,81.64,0.0,85.0,3.0,Corporate Auto,Four-Door Car,790.784983
7068,West Region,M,College,75.24,21941.0,96.0,0.0,Personal Auto,Four-Door Car,691.200000
